In [2]:
import numpy as np
import pandas as pd

import iqplot

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [50]:
df = pd.read_csv('data/grant_complete.csv', comment='#')

df.head()

,band,beak depth (mm),beak length (mm),species,year
0,20123,8.05,9.25,fortis,1973
1,20126,10.45,11.35,fortis,1973
2,20128,9.55,10.15,fortis,1973
3,20129,8.75,9.95,fortis,1973
4,20133,10.15,11.55,fortis,1973


In [51]:
df = df.loc[
    (df['species']=='scandens') & (df['year'].isin([1975, 2012])),
    ['year', 'beak depth (mm)', 'beak length (mm)']
]

In [5]:
df.head()

,year,beak depth (mm)
401,1975,8.4
402,1975,8.8
403,1975,8.4
404,1975,8.0
405,1975,7.9


In [6]:
p = iqplot.ecdf(
    data=df,
    q='beak depth (mm)',
    cats='year'
)

bokeh.io.show(p)

In [7]:
bd_1975 = df.loc[df['year']==1975, 'beak depth (mm)'].values
bd_2012 = df.loc[df['year']==2012, 'beak depth (mm)'].values

np.mean(bd_1975), np.mean(bd_2012)

(8.959999999999999, 9.188492063492063)

In [8]:
rng = np.random.default_rng()

In [20]:
np.mean(rng.normal(10, 1, size=10))

10.582303499877936

In [21]:
len(bd_1975)

87

In [22]:
rng = np.random.default_rng(3252)

bs_sample = rng.choice(bd_1975, replace=True, size=len(bd_1975))

In [24]:
p = iqplot.ecdf(
    data=bd_1975
)

p = iqplot.ecdf(
    data=bs_sample,
    style='staircase',
    p=p
)

bokeh.io.show(p)

In [25]:
bs_replicate = np.mean(bs_sample)

In [26]:
bs_replicate

8.849770114942531

In [30]:
def draw_bs_reps(data, func, rng, n_reps=1):
    """Draw a bootstrap replicate from data."""
    bs_reps = np.empty(n_reps)
    for i in range(n_reps):
        bs_sample = rng.choice(data, replace=True, size=len(data))
        bs_reps[i] = func(bs_sample)
        
    if n_reps == 1:
        return bs_reps[0]
    else:
        return bs_reps

In [43]:
n_reps = 100_000
bs_reps_1975 = draw_bs_reps(bd_1975, np.mean, rng, n_reps)
bs_reps_2012 = draw_bs_reps(bd_2012, np.mean, rng, n_reps)

conf_int_1975 = np.percentile(bs_reps_1975, [2.5, 97.5])
conf_int_2012 = np.percentile(bs_reps_2012, [2.5, 97.5])

In [44]:
conf_int_1975

array([8.84287356, 9.0791954 ])

In [45]:
conf_int_2012

array([9.07222222, 9.30595238])

In [47]:
years = ['2012', '1975']
p = bokeh.plotting.figure(
    frame_height=100,
    frame_width=250,
    x_axis_label='bead depth (mm)',
    y_range=years,
)

p.circle([np.mean(bd_2012), np.mean(bd_1975)], years, size=5)
p.line(conf_int_1975, ['1975']*2, line_width=3)
p.line(conf_int_2012, ['2012']*2, line_width=3)

bokeh.io.show(p)

In [49]:
np.percentile(bs_reps_2012 - bs_reps_1975, [0.5, 99.5])

array([0.00944394, 0.44752887])

In [36]:
np.std(bd_1975) / np.sqrt(len(bd_1975)) * 1.96

0.11837473262184112

In [53]:
p = bokeh.plotting.figure(
    frame_width=250,
    frame_height=250,
    x_axis_label='beak depth (mm)',
    y_axis_label='beak length (mm)',
)

bl_1975 = df.loc[df['year']==1975, 'beak length (mm)'].values

p.circle(bd_1975, bl_1975)

bokeh.io.show(p)

In [55]:
p = iqplot.ecdf(
    bl_1975 / bd_1975,
    x_axis_label='length/depth'
)

bokeh.io.show(p)

In [56]:
bs_reps_ratio = draw_bs_reps(bl_1975 / bd_1975, np.mean, rng, 100_000)

In [57]:
np.percentile(bs_reps_ratio, [2.5, 97.5])

array([1.56216016, 1.59585793])

In [68]:
def corr(x, y):



corr(bd_1975, bl_1975)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


0.6197221344707919

In [71]:
def draw_bs_pairs(x, y, rng):
    """Draw pairs of data points out of `x` and `y`."""
    bs_inds = rng.choice(np.arange(len(x)), replace=True, size=len(x))
    
    return x[bs_inds], y[bs_inds]

In [67]:
my_ar = np.array([1, 2, 3, 4, 5])

my_ar[np.array([2, 0, 2, 3, 4])]

array([3, 1, 3, 4, 5])

In [80]:
%%capture
n_reps = 10
corr_bs_reps = np.empty(n_reps)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i in range(n_reps):
        bd, bl = draw_bs_pairs(bd_1975, bl_1975, rng)
        corr_bs_reps[i] = np.corrcoef(bd, bl)

ValueError: setting an array element with a sequence.

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [75]:
warnings.filterwarnings?

Signature:
warnings.filterwarnings(
    action,
    message='',
    category=<class 'Warning'>,
    module='',
    lineno=0,
    append=False,
)
Docstring:
Insert an entry into the list of warnings filters (at the front).

'action' -- one of "error", "ignore", "always", "default", "module",
            or "once"
'message' -- a regex that the warning message must match
'category' -- a class that the warning must be a subclass of
'module' -- a regex that the module name must match
'lineno' -- an integer line number, 0 matches all warnings
'append' -- if true, append to the list of filters
File:      ~/opt/anaconda3/envs/bootcamp/lib/python3.11/warnings.py
Type:      function

In [82]:
p = iqplot.ecdf(
    data=df,
    q='beak depth (mm)',
    cats='year',
    style='staircase',
    conf_int=True,
)

bokeh.io.show(p)